In [1]:
# Langkah 1: Import libraries
import re
import csv
from pathlib import Path

In [2]:
# Langkah 2: Fungsi untuk ekstraksi TIME, IP dan URL
def extract_ip_url_time(log_line):
    # Pola regex untuk menangkap IP
    ip_pattern = r'(\d+\.\d+\.\d+\.\d+)'  
    # Pola regex untuk menangkap URL setelah GET
    url_pattern = r'"GET ([^"]+)'  
    # Pola regex untuk menangkap waktu dalam format 05/Sep/2024:06:32:48 +0700
    time_pattern = r'\[(.*?)\]'  
    
    # Cari pola IP, URL, dan waktu
    ip_match = re.search(ip_pattern, log_line)
    url_match = re.search(url_pattern, log_line)
    time_match = re.search(time_pattern, log_line)
    
    # Jika ditemukan, ambil nilainya; jika tidak, berikan string kosong
    ip = ip_match.group(1) if ip_match else ''
    url = url_match.group(1) if url_match else ''
    time = time_match.group(1) if time_match else ''
    
    # Hilangkan "HTTP/2.0" atau protokol HTTP lainnya dari URL jika ada
    url = url.split(' ')[0]  # Memotong bagian setelah spasi (misalnya "HTTP/2.0")
    
    # Hilangkan parameter query dari URL jika ada
    url = url.split('?')[0] if '?' in url else url
    
    return time, ip, url  # Mengembalikan waktu, IP, dan URL


In [3]:
# Langkah 3: Fungsi untuk memproses satu file log
def process_log_file(input_file):
    extracted_data = []  # List untuk menyimpan data waktu, IP, dan URL
    with open(input_file, 'r') as f:
        log_lines = f.readlines()  # Membaca semua baris di file log
    
    # Iterasi tiap baris log untuk mengekstraksi data
    for line in log_lines:
        time, ip, url = extract_ip_url_time(line)
        if ip and url:  # Jika IP dan URL valid, tambahkan ke list
            extracted_data.append((time, ip, url))
    
    return extracted_data  # Mengembalikan data yang diekstraksi


In [4]:
# Langkah 4: Fungsi untuk memproses semua file log dalam satu folder dan menyimpan hasil ke file CSV
def process_all_log_files(logs_folder, output_file):
    all_extracted_data = []  # List untuk menyimpan semua data waktu, IP, dan URL dari berbagai file log
    log_files = list(logs_folder.glob('response_time-simrs*'))  # Cari file log sesuai pola
    
    # Iterasi untuk memproses tiap file log
    for log_file in log_files:
        print(f"Processing file: {log_file}")
        extracted_data = process_log_file(log_file)  # Proses file log
        all_extracted_data.extend(extracted_data)  # Tambahkan data ke list total
    
    # Tulis hasil ke file CSV
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Time', 'IP', 'URL'])  # Header kolom dengan tambahan waktu
        writer.writerows(all_extracted_data)  # Menulis data yang diekstraksi

In [5]:
# Langkah 5: Bagian eksekusi utama, hanya jika script dijalankan langsung
if __name__ == '__main__':
    logs_folder = Path('../Data')  # Folder 'Data'
    output_file = 'extracted_data.csv'  # Nama file CSV keluaran

    # Memproses semua file log dan menyimpan hasilnya ke file CSV
    process_all_log_files(logs_folder, output_file)
    print(f"Data extracted and saved to {output_file}")

Processing file: ../Data/response_time-simrs.log.10
Processing file: ../Data/response_time-simrs.log.11
Processing file: ../Data/response_time-simrs.log.9
Processing file: ../Data/response_time-simrs.log.7
Processing file: ../Data/response_time-simrs.log.1
Processing file: ../Data/response_time-simrs.log.6
Processing file: ../Data/response_time-simrs.log.8
Processing file: ../Data/response_time-simrs.log.13
Processing file: ../Data/response_time-simrs.log.14
Processing file: ../Data/response_time-simrs.log.12
Processing file: ../Data/response_time-simrs.log
Processing file: ../Data/response_time-simrs.log.3
Processing file: ../Data/response_time-simrs.log.4
Processing file: ../Data/response_time-simrs.log.5
Processing file: ../Data/response_time-simrs.log.2
Data extracted and saved to extracted_data.csv
